# Retail weekly report

## *Version : Alpha 01*

1) Collecte des données
* Chargement des fichiers 
* Formattage en dataframe : zones et point de passage
* Création des données complémentaires (jours, numero de semaines, année)

2) Traitement
* Indicateurs globaux
* Comparatifs sites
* Donnée site de référence

3) Génération du rapport
* Indicateurs globaux
* Comparatifs sites
* Données par site

# 0) Initialisation

* Mise en place des librairies standard
* Chargement des modules utilisateurs
* Définition des variables globales

In [48]:
# Imports

import pandas as pd
import xlsxwriter as xw

from datetime import datetime

In [49]:
# Key constants

WEEK_DAYS = ['Lundi','Mardi','Mercredi','Jeudi','Vendredi','Samedi','Dimanche']
WEEK_DAYS_ABBR = ['Lun','Mar','Mer','Jeu','Ven','Sam','Dim']

RAW_DATA_PATH = "raw_data/"
MEDIA_PATH = "media/"
OUTPUT_PATH = "xlsx_output/"

ZONE_RAW_FILE = "Carrefour_hyper_KPIs_1-March-2021_to_1-June-2021_zone.xlsx"
PASSAGE_RAW_FILE = "Carrefour_hyper_KPIs_1-March-2021_to_1-June-2021_passage.xlsx"

# 1) Collecte des données

## 1.1 ) Chargement des données

Prérequis :
* Periode bien définie, commençant par un lundi et finissant par un dimanche, au moins deux semaines
* Contient tous les magasins du réseau à analyser

Fichiers :
* KPI sur periode par passage.xlsx
* KPI sur periode par zone.xlsx

Erreurs possibles :
* Non prise en compte d'erreurs de capture d'information / d'informations manquantes


In [50]:
# load passage & zone
df_passage_raw = pd.read_excel(RAW_DATA_PATH + PASSAGE_RAW_FILE) 
df_zone_raw = pd.read_excel(RAW_DATA_PATH + ZONE_RAW_FILE) 


## 2.2) Retraitements specifiques 

**TRAITEMENT SPECIAL 1 : renommer la colonne Date.1 en Hour**

In [51]:
# Date.1 -> Hour dans le titre de colonne

df_passage_raw.rename(columns = {'Date.1': 'Hour'}, inplace = True)

**TRAITEMENT SPECIAL : correction des fichiers démarrants à :40 ... (décalage vers 8:00 -> données fausses)**

In [52]:
# corrige / normalise les décalages horaires - attention pas normal d'avoir des slots qui commencent :40 ...
df_passage_raw['Hour'] = df_passage_raw['Hour'].str.replace(':40',':00')
df_zone_raw['Hour'] = df_zone_raw['Hour'].str.replace(':40',':00')

In [53]:
## 2.3) Création des champs utiles : Week, DayNb

Ajout d'une colonne année Year, mois Month, semaine Week, numéro du jour DayInWeek pour faire des regroupements

Les dates suivent la norme ISO 8601 : numérotation à partir de 01, lundi est le 1er jour de la semaine (donc jours vont de 1 à 7)

https://docs.python.org/3/library/datetime.html

https://stackoverflow.com/questions/2600775/how-to-get-week-number-in-python

https://docs.python.org/3/library/datetime.html#strftime-and-strptime-behavior

In [54]:
# constante pour isocalendar
YEAR_ISO = 0
WEEK_ISO = 1
DAYNB_ISO = 2
DATE_FORMAT = "%d/%m/%Y"

def get_iso_date_from_string(iso_info, string_date):
    a_date = datetime.strptime(string_date, DATE_FORMAT)
    return a_date.isocalendar()[iso_info]

def add_iso_date_col_to_df(df):
    new_df = df
    new_df['Year']= df['Date'].map(lambda x: get_iso_date_from_string(YEAR_ISO, x))
    new_df['Month']= df['Date'].map(lambda x: datetime.strptime(x, DATE_FORMAT).month)   
    new_df['Week']= df['Date'].map(lambda x: get_iso_date_from_string(WEEK_ISO, x))
    new_df['DayInWeek']= df['Date'].map(lambda x: get_iso_date_from_string(DAYNB_ISO, x))
    return new_df

In [55]:
df_passage_raw_2 = add_iso_date_col_to_df(df_passage_raw)
df_zone_raw_2 = add_iso_date_col_to_df(df_zone_raw)

In [56]:
df_passage_raw_2_columns = list(df_passage_raw_2.columns)
df_zone_raw_2_columns = list(df_zone_raw_2.columns)

In [58]:
list_of_sites_passage = df_passage_raw_2['Passage'].unique().tolist()
list_of_sites_zone = df_zone_raw_2['Zone'].unique().tolist()